In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [4]:
### Load the trained model, scaler and encoder
model = load_model('model.h5')

with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

with open('onehot_encoder_geo.pkl', 'rb') as f:
    onehot_encoder_geo = pickle.load(f)

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder_gender = pickle.load(f)

In [5]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [28]:
# Get geo onehot encoding with paramater sparse=False
geo_onehot = onehot_encoder_geo.transform([[input_data['Geography']]])
geo_df = pd.DataFrame(geo_onehot, columns=onehot_encoder_geo.get_feature_names_out())

c:\Users\Sasa\Documents\Programiranje\ann-project\venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [29]:
geo_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [31]:
input_df = pd.DataFrame([input_data])

In [33]:
## Combine the input data with the onehot encoding
input_data = pd.concat([input_df.reset_index(drop=True), geo_df], axis=1)

In [34]:
input_data.drop(columns=['Geography'], inplace=True)

In [35]:
input_data.head(1)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [36]:
input_df['Gender'] = label_encoder_gender.transform(input_df['Gender'])

In [37]:
input_df.head(1)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [38]:
input_data = pd.concat([input_df.reset_index(drop=True), geo_df], axis=1)

In [39]:
input_data.drop(columns=['Geography'], inplace=True)

In [40]:
input_data.head(1)

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [41]:
## Scaling the input data
input_scaled = scaler.transform(input_data)

In [42]:
## Prediction
pred = model.predict(input_scaled)

1/1 [==============================] - 0s 200ms/step


In [43]:
pred

array([[0.04194679]], dtype=float32)

In [44]:
pred_probabilty = pred[0][0]

In [45]:
pred_probabilty

0.041946795

In [46]:
if pred_probabilty > 0.5:
    print('The customer is likely to churn')
else:
    print('The customer is not likely to churn')

The customer is not likely to churn
